In [24]:
#https://www.kaggle.com/code/grvychuck/spaceship-titanic-randomforest

import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings


final = 'final_space.csv'
df = pd.read_csv(final)
submit = pd.read_csv('Age_test.csv')

x =df.drop(['PassengerId', 'Transported'], axis=1)
y =df['Transported']

def get_train_test_dataset(df=None):
    df_copy = df.copy()


    # train_test_split( )으로 학습과 테스트 데이터 분할.
    X_train, X_test, y_train, y_test = \
    train_test_split(x,y,test_size=0.3, random_state=0, stratify=y)

    return X_train, X_test, y_train, y_test




X_train, X_test, y_train, y_test = get_train_test_dataset(df)
print('학습 데이터 레이블 값 비율')

print(y_train.value_counts()/y_train.shape[0] * 100)
print('테스트 데이터 레이블 값 비율')
print(y_test.value_counts()/y_test.shape[0] * 100)

학습 데이터 레이블 값 비율
1    50.369762
0    49.630238
Name: Transported, dtype: float64
테스트 데이터 레이블 값 비율
1    50.345092
0    49.654908
Name: Transported, dtype: float64


In [25]:

from sklearn.model_selection import GridSearchCV

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


# 랜덤 포레스트 학습 및 별도의 테스트 셋으로 예측 성능 평가



rf_clf = RandomForestClassifier(random_state=0)
rf_clf.fit(X_train , y_train)
pred = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test , pred)
print('랜덤 포레스트 정확도: {0:.4f}'.format(accuracy))

랜덤 포레스트 정확도: 0.7926


In [27]:
params = {
    'n_estimators':[100,110],
    'max_depth' : [8, 10, 12,14], 
    'min_samples_leaf' : [4,6,8],
    'min_samples_split' : [3,4,5]
}
# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf , param_grid=params , cv=4, n_jobs=-1 )
grid_cv.fit(X_train , y_train)

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:
 {'max_depth': 14, 'min_samples_leaf': 8, 'min_samples_split': 3, 'n_estimators': 110}
최고 예측 정확도: 0.8067


In [28]:
for dataset in [submit]:
   
    dataset['RoomService'] = dataset['RoomService'].fillna(dataset['RoomService'].median())
    dataset['FoodCourt'] = dataset['FoodCourt'].fillna(dataset['FoodCourt'].median())
    dataset['Spa'] = dataset['Spa'].fillna(dataset['Spa'].median())
    dataset['ShoppingMall'] = dataset['ShoppingMall'].fillna(dataset['ShoppingMall'].median())
    dataset['VRDeck'] = dataset['VRDeck'].fillna(dataset['VRDeck'].median())
    dataset['Cabin_number'] = dataset['Cabin_number'].fillna(dataset['Cabin_number'].median())

In [29]:


model = RandomForestClassifier(n_estimators = 110, random_state=0, max_depth = 14, min_samples_leaf = 8, min_samples_split = 3, oob_score=True)
X_test = submit.drop('PassengerId', axis=1)

model.fit(X_train, y_train)
submission2 = pd.DataFrame({
        "PassengerId": submit["PassengerId"],
        "Transported": model.predict(X_test).astype('bool')
    })

submission2.to_csv('submission2.csv', index=False)